In [23]:
from sklearn.feature_selection import mutual_info_regression, f_regression
from sklearn.feature_selection import SelectPercentile
from sklearn.linear_model import RandomizedLasso, Ridge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
from subprocess import check_output


In [3]:
df = pd.read_csv(r'/home/pirwin/Git/IPy_Notebooks/SQL/CNTYVARS.csv')
#df = pd.read_csv(r'C:\Users\pairwin\Documents\GitHub\IPy_Notebooks\SQL\CNTYVARS.csv')
df['HSGRAD_PRCNT'] = df['HSGRAD_CY']/df['TOTPOP_CY']

In [4]:
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
print(dtype_df[dtype_df['Column Type'] == 'object'])

         Count Column Type
783   TSEGCODE      object
784   TSEGNAME      object
854  TLIFECODE      object
855  TLIFENAME      object


In [5]:
dp = ['OBJECTID','Customers', 'UNITS', 'SALES', 'LOG_UNITS', 'TSEGCODE','TSEGNAME','TLIFECODE','TLIFENAME']

X = df.drop(dp, axis=1)

#X2 = transformer.fit_transform(X, y)

#X2.head()

In [6]:
y = df[['LOG_UNITS']]
y.shape

(3109, 1)

In [7]:
selector = SelectPercentile(f_regression, percentile=20)
columns = X.columns
selector.fit_transform(X, np.array(y))
labels = [columns[x] for x in selector.get_support(indices=True) if x]
XNew = pd.DataFrame(selector.fit_transform(X, np.array(y)), columns = labels)
columns = XNew.columns


lst = list()
for n, s in zip(columns, selector.scores_):
    lst.append((s, n))

scores = pd.DataFrame(lst, columns=['Score','Name'])
    


/usr/lib/python3/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/lib/python3/dist-packages/sklearn/feature_selection/univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
/home/pirwin/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/pirwin/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/pirwin/.local/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [8]:
inds = list()
for val in columns:
    #print(val)
    inds.append((val, XNew.columns.get_loc(val)))
    
indices = pd.DataFrame(inds, columns=['Name','XNew_Index'])
    


In [9]:
scores.sort_values(by='Score',inplace=True, ascending=True)
scores.reset_index(inplace=True)
#scores.to_csv(r'C:\Users\pairwin\Documents\GitHub\IPy_Notebooks\SQL\varscores.csv')
#scores.to_csv(r'/home/pirwin/Git/IPy_Notebooks/SQL/varscores.csv')

In [10]:
scores.drop('index', inplace=True, axis = 1)

df = pd.merge(scores, indices, how='inner',left_on='Name',right_on='Name')



In [12]:
def calculate_vif(X, thresh=7.5):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            # Loop repeatedly until we find that all columns within our dataset
            # have a VIF value we're happy with.
            variables = X.columns
            dropped=False
            vif = []
            new_vif = 0
            for var in variables:
                new_vif = variance_inflation_factor(X.as_matrix(), X.columns.get_loc(var))
                vif.append(new_vif)
                if np.isinf(new_vif):
                    break
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)                
                #print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                #df = df[df['Name'] != str(X.columns[maxloc])]
                dropped=True
        return X

In [ ]:
finaldf = calculate_vif(XNew)

In [34]:
finaldf.tail()

,INDPUBL_CY,VAL0_CY,VAL100K_CY,THHGRPL4,THHGRPL5,THHGRPL6,THHGRPU5,THHGRPU6,TADULT21,TADULT38,S31_EMP,N04_BUS,N36_BUS
3104,191.0,346.0,419.0,0.0,0.0,1944.0,992.0,1871.0,0.0,0.0,282.0,1.0,7.0
3105,2454.0,1081.0,1807.0,3647.0,3310.0,8228.0,4306.0,7802.0,9487.0,737.0,1630.0,7.0,32.0
3106,248.0,137.0,234.0,0.0,0.0,2093.0,0.0,2598.0,0.0,0.0,293.0,1.0,23.0
3107,423.0,430.0,788.0,855.0,4080.0,2032.0,1348.0,3336.0,0.0,1555.0,722.0,9.0,57.0
3108,778.0,621.0,1855.0,0.0,0.0,7934.0,3952.0,7939.0,0.0,0.0,1566.0,5.0,33.0


In [13]:
rlasso = RandomizedLasso(alpha=0.001)
rlasso.fit(X, np.ravel(y))

/usr/lib/python3/dist-packages/sklearn/linear_model/least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))


RandomizedLasso(alpha=0.001, eps=2.2204460492503131e-16, fit_intercept=True,
        max_iter=500, memory=Memory(cachedir=None), n_jobs=1,
        n_resampling=200, normalize=True, pre_dispatch='3*n_jobs',
        precompute='auto', random_state=None, sample_fraction=0.75,
        scaling=0.5, selection_threshold=0.25, verbose=False)

In [14]:
print("Features sorted by their score:")
lassoscored = pd.DataFrame(sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), 
                 X.columns), reverse=True), columns=['Score','Feature'])
lassoscored = lassoscored[lassoscored['Score'] > 0]
lassolist = lassoscored['Feature'].as_matrix().tolist()
print(lassoscored)

Features sorted by their score:
     Score     Feature
0    0.870  MEDNWA15CY
1    0.780    THHGRPU6
2    0.740       THH47
3    0.735   MEDHHR_CY
4    0.725    LANDAREA
5    0.680   MEDVAL_CY
6    0.610    TADULT58
7    0.600       THH24
8    0.565  UNEMPRT_CY
9    0.515     N40_BUS
10   0.490    THHGRPU5
11   0.490  AVGNWA65CY
12   0.485  AVGNWA35CY
13   0.455   THHGRPL10
14   0.425       THH48
15   0.425  AVGNWA45CY
16   0.415  AVGNWA55CY
17   0.380  POPGRW10CY
18   0.355    TADULT18
19   0.355  MEDMAGE_CY
20   0.315       THH58
21   0.310       THH22
22   0.300       THH13
23   0.295    TADULT47
24   0.290       THH25
25   0.280   AVGVAL_CY
26   0.265  AVGDIA15CY
27   0.255    TADULT43
28   0.255  INDPUBL_CY
29   0.250    TADULT24
..     ...         ...
128  0.010     N18_EMP
129  0.010     N15_EMP
130  0.010   N03_SALES
131  0.010  MEDDIA65CY
132  0.010  AVGHHSZ_CY
133  0.010  AIFBASE_CY
134  0.005     TSEGNUM
135  0.005   THHGRPL12
136  0.005       THH55
137  0.005       THH51
13

In [25]:
ridgereg = Ridge(alpha=.001, normalize=True)
ridgereg.fit(X, np.ravel(y))
ridgereg.

<bound method RegressorMixin.score of Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)>

In [15]:
XNew2 = X[lassolist]

In [16]:
XNew2.head()

,MEDNWA15CY,THHGRPU6,THH47,MEDHHR_CY,LANDAREA,MEDVAL_CY,TADULT58,THH24,UNEMPRT_CY,N40_BUS,...,N33_SALES,MEDNWA75CY,MEDNWA25CY,MEDIA35_CY,AVGNWA75CY,A75NW35_CY,A75NW150CY,A75I25_CY,A65DI35_CY,A65DI15_CY
0,10426,856,0,50.3,739.0174,590884,2288,0,6.4,1079,...,10099285,226658,15020,86395,743687,819,6610,5782,9266,7475
1,13472,7783,0,60.4,594.5831,284436,872,0,8.0,110,...,133723,249707,17737,70258,852977,36,519,476,576,449
2,8933,9415,1609,53.5,1636.4644,259717,2415,830,8.7,272,...,1946770,204589,11450,54101,634734,204,2156,1827,2436,2127
3,15574,10535,0,59.9,1020.0123,256835,1442,0,5.3,105,...,172631,250001,16458,67159,883979,36,549,485,639,569
4,11583,1744,0,52.1,1150.7310,214455,539,594,7.0,126,...,114115,198987,13206,54234,528781,15,155,150,175,165


In [17]:
lassofinal = calculate_vif(XNew2)

/home/pirwin/.local/lib/python3.5/site-packages/statsmodels/stats/outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [18]:
lassofinal.head()

,LANDAREA,MEDVAL_CY,TADULT58,UNEMPRT_CY,THH48,TADULT18,THH13,TADULT47,TADULT43,TADULT24,...,TADULT41,NHSPOTH_CY,N15_EMP,N03_SALES,THH55,THH51,THH11,OCCFARM_CY,NHSPAI_CY,MEDNWA25CY
0,739.0174,590884,2288,6.4,0,0,4102,0,7119,0,...,6109,4174,1597,30207,4429,7733,12866,1854,4034,15020
1,594.5831,284436,872,8.0,0,0,0,0,3125,0,...,717,121,75,12000,0,0,0,36,562,17737
2,1636.4644,259717,2415,8.7,334,1611,0,3596,29493,1773,...,3314,307,222,34393,0,0,0,1723,3454,11450
3,1020.0123,256835,1442,5.3,0,0,0,0,6452,0,...,0,68,51,4014,0,0,0,145,527,16458
4,1150.7310,214455,539,7.0,0,0,0,0,564,1206,...,0,29,55,3840,0,0,0,1652,262,13206


In [19]:
import statsmodels.api as sts


/home/pirwin/.local/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [22]:
XNew3 = sts.add_constant(XNew2)
mod = sts.OLS(y, XNew3)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              LOG_UNITS   R-squared:                       0.758
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     60.87
Date:                Wed, 06 Sep 2017   Prob (F-statistic):               0.00
Time:                        23:50:09   Log-Likelihood:                -3272.1
No. Observations:                3109   AIC:                             6850.
Df Residuals:                    2956   BIC:                             7775.
Df Model:                         152                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           12.9595      0.880     14.727   